# 使用pytorch 自动求导

    2.5.1 自动求导要点
    2.5.2计算图
    2.5.3 标量反向传播
    2.5.4 非标量反向传播

In [1]:
import torch

x = torch.randn(3,4,requires_grad=True)
print(x)

# 使用 requires_grad 参数
# x.requires_grad = True

y  = torch.randn(3,4 ,requires_grad=True)
print(y)

t = x + y
z = t.sum()
print(z)

z.backward()
print(y.grad)

print(x.requires_grad, y.requires_grad, z.requires_grad)


tensor([[-0.6043, -0.1153,  0.3664, -0.4803],
        [-0.8925, -0.4327, -0.4175,  0.6769],
        [ 0.3795,  1.0685, -1.3675,  1.3260]], requires_grad=True)
tensor([[ 1.0355, -1.0343, -0.3214,  0.2042],
        [ 1.3049,  0.4144,  0.7120,  1.8672],
        [ 1.6937,  0.6042,  1.2528, -0.0939]], requires_grad=True)
tensor(7.1465, grad_fn=<SumBackward0>)
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
True True True


In [2]:
# 举个例子  z = w * x +b
x = torch.randn(1)
b = torch.randn(1, requires_grad=True)
w = torch.randn(1, requires_grad=True)
y = w * x
z = y + b
# 查看tensor是否需要求导，计算梯度
print(x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad)
# 查看 是否是叶子结点
print(x.is_leaf, b.is_leaf, w.is_leaf, y.is_leaf,z.is_leaf)

False True True True
True True True False False


In [5]:
# 反向传播计算
z.backward(retain_graph = True)
print('w.grad:', w.grad)
print('b.grad:', b.grad) # 梯度会累加


## 2.5 Tensor与Autograd
> 参数学习离不开求导，Pytorch是如何进行求导的呢？

autograd包为张量上所有的操作提供了自动求导功能，
而torch.Tensor和torch.Function为autograd上的两个核心类，他们相互连接并生成一个有向非循环图

### 2.5.1 自动求导要点

autograd包为对tensor进行自动求导，需考虑如下事项：

（1）创建叶子节点(leaf node)的tensor，使用requires_grad参数指定是否记录对其的操作，以便之后利用backward()方法进行梯度求解。requires_grad参数缺省值为False，如果要对其求导需设置为True，与之有依赖关系的节点自动变为True。

（2）可利用requires_grad_()方法修改tensor的requires_grad属性。可以调用.detach()或with torch.no_grad():将不再计算张量的梯度，跟踪张量的历史记录。这点在评估模型、测试模型阶段常常使用。

（3）通过运算创建的tensor（即非叶子节点），会自动被赋于grad_fn属性。该属性表示梯度函数。叶子节点的grad_fn为None。

（4）最后得到的tensor执行backward()函数，此时自动计算各变在量的梯度，并将累加结果保存grad属性中。计算完成后，非叶子节点的梯度自动释放。

（5）backward()函数接受参数，该参数应和调用backward()函数的Tensor的维度相同，或者是可broadcast的维度。如果求导的tensor为标量（即一个数字），backward中参数可省略。

（6）反向传播的中间缓存会被清空，如果需要进行多次反向传播，需要指定backward中的参数retain_graph=True。多次反向传播时，梯度是累加的。

（7）非叶子节点的梯度backward调用后即被清空。

（8）可以通过用torch.no_grad()包裹代码块来阻止autograd去跟踪那些标记为.requesgrad=True的张量的历史记录。

> 整个过程中，Pytorch采用计算图的形式进行组织，该计算图为动态图，它的计算图在每次前向传播时，将重新构建

### 2.5.2计算图

计算图是一种有向无环图像，用图形方式表示算子与变量之间的关系

### 2.5.3 标量反向传播

假设x、w、b都是标量，z=wx+b，对标量z调用backward()

w.grad: tensor([-2.1708])
b.grad: tensor([3.])


In [ ]:
import torch

## (1)定义叶子节点及算子节点
# 定义输入张量x
x=torch.Tensor([2])
#初始化权重参数W,偏移量b、并设置require_grad属性为True，为自动求导t
w=torch.randn(1,requires_grad=True)
b=torch.randn(1,requires_grad=True)
#实现前向传播
y=torch.mul(w,x)  #等价于w*x
z=torch.add(y,b)  #等价于y+b
#查看x,w，b 叶子节点的requite_grad属性: False,True,True
print("x,w,b的require_grad属性分别为：{},{},{}".format(x.requires_grad,w.requires_grad,b.requires_grad))

# （2）查看叶子节点、非叶子节点的其他属性

#查看非叶子节点的requres_grad属性,
print("y，z的requires_grad属性分别为：{},{}".format(y.requires_grad,z.requires_grad))
#因与w，b有依赖关系，故y，z的requires_grad属性也是：True,True
#查看各节点是否为叶子节点
print("x，w，b，y，z的是否为叶子节点：{},{},{},{},{}".format(x.is_leaf,w.is_leaf,b.is_leaf,y.is_leaf,z.is_leaf))
#x，w，b，y，z的是否为叶子节点：True,True,True,False,False
#查看叶子节点的grad_fn属性
print("x，w，b的grad_fn属性：{},{},{}".format(x.grad_fn,w.grad_fn,b.grad_fn))
#因x，w，b为用户创建的，为通过其他张量计算得到，故x，w，b的grad_fn属性：None,None,None
#查看非叶子节点的grad_fn属性
print("y，z的是否为叶子节点：{},{}".format(y.grad_fn,z.grad_fn))
#y，z的是否为叶子节点：,

# （3）自动求导，实现梯度方向传播，即梯度的反向传播。

# 基于z张量进行梯度反向传播,执行backward之后计算图会自动清空，
z.backward()
#如果需要多次使用backward，需要修改参数retain_graph为True，此时梯度是累加的
#z.backward(retain_graph=True)

#查看叶子节点的梯度，x是叶子节点但它无需求导，故其梯度为None
print("参数w,b的梯度分别为:{},{},{}".format(w.grad,b.grad,x.grad))
#参数w,b的梯度分别为:tensor([2.]),tensor([1.]),None

#非叶子节点的梯度，执行backward之后，会自动清空
print("非叶子节点y,z的梯度分别为:{},{}".format(y.grad,z.grad))
#非叶子节点y,z的梯度分别为:None,None


### 2.5.4 非标量反向传播

Pytorch有个简单的规定，不让张量(tensor)对张量求导，只允许标量对张量求导

如果目标张量对一个非标量调用backward()，需要传入一个gradient参数,该参数也是张量，而且需要与调用backward()的张量形状相同。

为什么要传入一个张量gradient？
传入这个参数就是为了把张量对张量求导转换为标量对张量求导。

假设目标值为loss=(y_1,y_2,…,y_m)传入的参数为v=(v_1,v_2,…,v_m),那么就可把对loss的求导，转换为对loss*v^T标量的求导。即把原来∂loss/∂x得到雅可比矩阵(Jacobian)乘以张量v^T，便可得到我们需要的梯度矩阵。

backward函数的格式为： `backward(gradient=None, retain_graph=None, create_graph=False)`


 （1）定义叶子叶子节点及计算节点

In [ ]:
import torch

#定义叶子节点张量x，形状为1x2
x= torch.tensor([[2, 3]], dtype=torch.float, requires_grad=True)
#初始化Jacobian矩阵
J= torch.zeros(2 ,2)
#初始化目标张量，形状为1x2
y = torch.zeros(1, 2)
#定义y与x之间的映射关系：
#y1=x1**2+3*x2，y2=x2**2+2*x1
y[0, 0] = x[0, 0] ** 2 + 3 * x[0 ,1]
y[0, 1] = x[0, 1] ** 2 + 2 * x[0, 0]

（2）手工计算y对x的梯度

![](./cal_acobian.png)

（2）调用backward获取y对x的梯度


In [ ]:
# y.backward(torch.Tensor([[1, 1]]))
# print(x.grad)
# 结果为tensor([[6., 9.]])
# 错在v的取值错误，通过这种方式得的到并不是y对x的梯度。这里我们可以分成两步的计算。首先让v=(1,0)得到y_1对x的梯度，然后使v=(0,1)，得到y_2对x的梯度

#生成y1对x的梯度
y.backward(torch.Tensor([[1, 0]]),retain_graph=True)
J[0]=x.grad
#梯度是累加的，故需要对x的梯度清零
x.grad = torch.zeros_like(x.grad)
#生成y2对x的梯度
y.backward(torch.Tensor([[0, 1]]))
J[1]=x.grad
print(J)

## 2.6 使用Numpy实现机器学习

1. 给出一个数组x，然后基于表达式：y=3x^2+2，加上一些噪音数据到达另一组数据y。
2. 构建一个机器学习模型，学习表达式y=wx^2+b的两个参数w，b。利用数组x，y的数据为训练数据
3. 采用梯度梯度下降法，通过多次迭代，学习到w、b的值

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
# 生成输入数据x及目标数据y
np.random.seed(100)
x = np.linspace(-1, 1, 100).reshape(100,1)
y = 3*np.power(x, 2) +2+ 0.2*np.random.rand(x.size).reshape(100,1)

# 画图
plt.scatter(x, y)
plt.show()

训练模型 - 定义损失函数，假设批量大小为100：

![](ch02_loss.png)

In [ ]:
# 随机初始化参数
w1 = np.random.rand(1,1)
b1 = np.random.rand(1,1)

# 训练模型
lr =0.001 # 学习率

for i in range(800):
    # 前向传播
    y_pred = np.power(x,2)*w1 + b1
    # 定义损失函数
    loss = 0.5 * (y_pred - y) ** 2
    loss = loss.sum()
    #计算梯度
    grad_w=np.sum((y_pred - y)*np.power(x,2))
    grad_b=np.sum((y_pred - y))
    #使用梯度下降法，是loss最小
    w1 -= lr * grad_w
    b1 -= lr * grad_b

plt.plot(x, y_pred,'r-',label='predict')
plt.scatter(x, y,color='blue',marker='o',label='true') # true data
plt.xlim(-1,1)
plt.ylim(2,6)
plt.legend()
plt.show()
print(w1,b1)

## 2.7 使用Tensor及antograd实现机器学习

In [ ]:
import torch
# 准备数据
torch.manual_seed(100)
#生成x坐标数据，x为tenor，需要把x的形状转换为100x1
x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)
#生成y坐标数据，y为tenor，形状为100x1，另加上一些噪音
y = 3*x.pow(2) +2+ 0.2*torch.rand(x.size())

# 画图，把tensor数据转换为numpy数据
plt.scatter(x.numpy(), y.numpy())
plt.show()


# 初始化权重参数

# 随机初始化参数，参数w，b为需要学习的，故需requires_grad=True
w = torch.randn(1,1, dtype=torch.float,requires_grad=True)
b = torch.zeros(1,1, dtype=torch.float, requires_grad=True)

# 训练模型
lr =0.001 # 学习率
for ii in range(800):
    # 前向传播，并定义损失函数loss
    y_pred = x.pow(2).mm(w) + b
    loss = 0.5 * (y_pred - y) ** 2
    loss = loss.sum()

    # 自动计算梯度，梯度存放在grad属性中
    loss.backward()

    # 手动更新参数，需要用torch.no_grad()，使上下文环境中切断自动求导的计算
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad

    # 梯度清零
        w.grad.zero_()
        b.grad.zero_()
# 可视化训练结果
plt.plot(x.numpy(), y_pred.detach().numpy(),'r-',label='predict')#predict
plt.scatter(x.numpy(), y.numpy(),color='blue',marker='o',label='true') # true data
plt.xlim(-1,1)
plt.ylim(2,6)
plt.legend()
plt.show()

print(w, b)
